In [ ]:
# Torch imports
import torch
import torchaudio
from torch import nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from pytorch_tcn import TCN

# Metrics and visualization
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns

# Models and feature extractor
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# Others
import os
from tqdm import tqdm
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D  # Necesario para crear la leyenda a medida
import pandas as pd
from glob import glob

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


architectures = ["GRU", "LSTM", "RNN", "CNN", "TCNN", "Transformer"]
color_map = {
    arch: default_colors[i % len(default_colors)]
    for i, arch in enumerate(architectures)
}


## Datasets

In [ ]:
class CustomSpeechCommands(Dataset):
    def __init__(self, root, files_list, download=True, target_len=16000, mode="mfcc", cnn_model=None):
        """
        mode: 'mfcc', 'mfcc_delta', 'mfcc_delta_delta', 'cnn', 'wav2vec2'
        cnn_model: modelo CNN preentrenado o personalizado para extracción
        """
        self.target_len = target_len
        self.mode = mode
        self.cnn_model = cnn_model
        self.dataset = torchaudio.datasets.SPEECHCOMMANDS(root=root, download=download)
        self.indices = None
        self.splitter(files_list, root)
        all_labels = sorted(list(set([label for _,_,label,_,_ in self.dataset])))
        self.label_to_idx = {l: i for i, l in enumerate(all_labels)}


    def splitter(self, files_list, root):
        with open(files_list, 'r') as f:
            self.file_paths = [line.strip() for line in f.readlines()]

        self.all_paths = []
        for item in tqdm(self.dataset._walker, desc=f"Splitting {files_list}"):
            relative_path = os.path.relpath(
                item,
                start=os.path.join(root, "SpeechCommands", "speech_commands_v0.02")
            ).replace("\\", "/")
            self.all_paths.append(relative_path)

        self.indices = [i for i, path in enumerate(self.all_paths) if path in self.file_paths]
        print(f"Archivos encontrados: {len(self.indices)} / {len(self.file_paths)}")

    def pad_waveform(self, waveform):
        length = waveform.shape[-1]
        if length < self.target_len:
            waveform = F.pad(waveform, (0, self.target_len - length))
        elif length > self.target_len:
            waveform = waveform[:, :self.target_len]
        return waveform

    def extract_feature_single(self, waveform, sample_rate, feature_extractor=None, processor=None, device="cuda"):
        """
        Extrae features de UNA muestra según el modo configurado.
        """
        waveform = self.pad_waveform(waveform).to(device)

        if feature_extractor is not None:
            feature_extractor = feature_extractor.to(device)

        # --- MFCC ---
        if self.mode == "mfcc":
            feat = feature_extractor(waveform).squeeze(0).cpu().transpose(0, 1)

        # --- MFCC + Delta ---
        elif self.mode == "mfcc_delta":
            base = feature_extractor(waveform)
            delta = torchaudio.functional.compute_deltas(base)
            feat = torch.cat([base, delta], dim=1).squeeze(0).cpu().transpose(0, 1)

        # --- MFCC + Delta + Delta-Delta ---
        elif self.mode == "mfcc_delta_delta":
            base = feature_extractor(waveform)
            delta = torchaudio.functional.compute_deltas(base)
            delta2 = torchaudio.functional.compute_deltas(delta)
            feat = torch.cat([base, delta, delta2], dim=1).squeeze(0).cpu().transpose(0, 1)

        # --- CNN ---
        elif self.mode == "cnn":
            spec_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate).to(device)
            spec = spec_transform(waveform).unsqueeze(0)
            with torch.no_grad():
                embedding = self.cnn_model(spec.to(device)).cpu().squeeze()
            feat = embedding

        # --- Wav2Vec2 ---
        elif self.mode == "wav2vec2":
            waveform = waveform.squeeze(0)
            inputs = processor(
                waveform,
                sampling_rate=sample_rate,
                return_tensors="pt",
                padding=True
            ).to(device)
            with torch.no_grad():
                outputs = feature_extractor(**inputs)
            feat = outputs.last_hidden_state.squeeze(0).cpu()

        else:
            raise ValueError(f"Modo de extracción '{self.mode}' no soportado.")

        return feat

    def extract_features(self, feature_extractor=None, processor=None, device="cuda"):
        features, labels = [], []

        with torch.no_grad():
            for idx in tqdm(self.indices, desc=f"Extrayendo features ({self.mode})"):
                waveform, sample_rate, label, _, _ = self.dataset[idx]
                feat = self.extract_feature_single(
                    waveform, sample_rate, feature_extractor, processor, device
                )
                features.append(feat)
                labels.append(label)

        features = torch.stack(features)
        print(f"Features tensor: {features.shape}")

        return features, labels

    def save_features(self, feature_extractor=None, save_path=None, processor=None, device="cuda"):
        print(f"Guardando features ({self.mode}) en {save_path}")
        try:
            features, labels = self.extract_features(feature_extractor, processor, device)
            torch.save({"features": features, "labels": labels}, save_path)
            print(f"Features guardadas correctamente en {save_path}")
            print(f"Clases finales: {set(labels)}")
        except Exception as e:
            print(f"Error al guardar features en {save_path}: {e}")

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        waveform, sample_rate, label, speaker_id, utterance_number = self.dataset[original_idx]
        waveform = self.pad_waveform(waveform)
        label_idx = self.label_to_idx[label]
        return waveform, label_idx


class FeaturesDataset(Dataset):
    def __init__(self, features_path):
        """
        Carga un archivo .pt con 'features' y 'labels' previamente guardados.

        features_path: ruta al archivo .pt (por ejemplo 'data/train.pt')
        """
        data = torch.load(features_path)
        self.features = data["features"]
        self.labels = data["labels"]

        self.label_to_idx = {label: i for i, label in enumerate(sorted(set(self.labels)))}
        self.idx_to_label = {v: k for k, v in self.label_to_idx.items()}
        self.numeric_labels = torch.tensor([self.label_to_idx[l] for l in self.labels])

        print(f"Dataset cargado desde {features_path}")
        print(f" - {len(self.features)} ejemplos")
        print(f" - {len(self.label_to_idx)} clases")

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.numeric_labels[idx]
        return feature, label


## Models

In [ ]:
class RNNModel(nn.Module):
    def __init__(
        self,
        rnn_type,
        n_input_channels,
        hidd_size=256,
        out_features = 35,
        num_layers=1,
    ):
        """
        Para utilizar una vanilla RNN entregue rnn_type="RNN"
        Para utilizar una LSTM entregue rnn_type="LSTM"
        Para utilizar una GRU entregue rnn_type="GRU"
        """
        super().__init__()

        self.rnn_type = rnn_type

        if rnn_type == "GRU":
            self.rnn_layer = nn.GRU(n_input_channels, hidd_size, batch_first=True, num_layers=num_layers)

        elif rnn_type == "LSTM":
            self.rnn_layer = nn.LSTM(n_input_channels, hidd_size, batch_first=True, num_layers=num_layers)

        elif rnn_type == "RNN":
            self.rnn_layer = nn.RNN(n_input_channels, hidd_size, batch_first=True, num_layers=num_layers, bidirectional=True)

        else:
            raise ValueError(f"rnn_type {rnn_type} not supported.")

        self.net = nn.Sequential(
            nn.Linear(hidd_size, out_features),
        )

        self.flatten_layer = nn.Flatten()

    def forward(self, x):
        if self.rnn_type == "GRU":
            out, h = self.rnn_layer(x)

        elif self.rnn_type == "LSTM":
            out, (h, c) = self.rnn_layer(x)

        elif self.rnn_type == "RNN":
            out, h = self.rnn_layer(x)

        out = h[-1]

        return self.net(out)

class CNNModel(nn.Module):
    def __init__(self, n_input_channels, hidd_size=64, out_features=35):
        """
        Modelo CNN (Convolutional Neural Network)

        Args:
            n_input_channels (int): Canales de entrada (e.g., 13 para MFCC)
            hidd_size (int): Número base de canales en las capas convolucionales
            out_features (int): Número de clases de salida (e.g., 35)
        """
        super().__init__()

        # --- Bloques Convolucionales ---
        # nn.Conv1d espera la entrada como (Batch, Channels, SeqLen)
        
        # (B, 13, T) -> (B, 64, T/2)
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(n_input_channels, hidd_size, kernel_size=5, padding=2),
            nn.BatchNorm1d(hidd_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        # (B, 64, T/2) -> (B, 128, T/4)
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(hidd_size, hidd_size * 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidd_size * 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        # (B, 128, T/4) -> (B, 256, T/4)
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(hidd_size * 2, hidd_size * 4, kernel_size=3, padding=1),
            nn.BatchNorm1d(hidd_size * 4),
            nn.ReLU(),
        )

        # --- Pooling Global y Clasificación ---
        
        # Colapsa la dimensión de secuencia (T/4) a 1
        # (B, 256, T/4) -> (B, 256, 1)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        self.flatten = nn.Flatten()
        
        # (B, 256) -> (B, 35)
        self.fc = nn.Linear(hidd_size * 4, out_features)

    def forward(self, x):
        x = x.permute(0, 2, 1) 
        
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.global_pool(x)
        x = self.flatten(x)
        
        # Clasificar
        return self.fc(x)

class PositionalEncoding(nn.Module):
    """
    Implementa el Positional Encoding para añadir información de posición.
    """
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Matriz de Positional Encoding
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [BatchSize, SeqLen, EmbeddingDim]
        """
        x = x + self.pe[:x.size(1)].transpose(0, 1) # Transpose para hacer Broadcasting [1, SeqLen, EmbDim]
        return self.dropout(x)


class TransformerModel(nn.Module):
    def __init__(
        self,
        n_input_features: int,
        n_output_classes: int = 35,
        d_model: int = 128,
        nhead: int = 8,
        d_hid: int = 256,
        n_layers: int = 6,
        dropout: float = 0.1
    ):
        super().__init__()
        
        self.model_type = 'Transformer'
        self.d_model = d_model
        
        self.input_projection = nn.Linear(n_input_features, d_model)
        
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=d_hid, 
            dropout=dropout,
            batch_first=True 
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        
        self.classifier = nn.Sequential(
            nn.Linear(d_model, n_output_classes)
        )
        
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.input_projection.bias.data.zero_()
        self.input_projection.weight.data.uniform_(-initrange, initrange)
        self.classifier[0].bias.data.zero_()
        self.classifier[0].weight.data.uniform_(-initrange, initrange)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [BatchSize, SeqLen, n_input_features]
        """
        x = self.input_projection(x) * np.sqrt(self.d_model) # Factor de escalamiento
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x) 
        final_representation = output[:, 0, :] 
        
        return self.classifier(final_representation)



class CNN1DModel(nn.Module):
    def __init__(
        self,
        hidd_size=256,
        in_channels=1,
        conv_out_channels=128,
        out_features=35,
        N_conv_blocks=4,
    ):
        super().__init__()

        # Primera capa estilo MFCC
        self.conv_blocks = nn.Sequential(
            nn.Conv1d(in_channels, conv_out_channels,
                      kernel_size=320, stride=160),
            nn.ReLU()
        )
        aux_channels = conv_out_channels
        for _ in range(N_conv_blocks-1):
            block = nn.Sequential(
                nn.Conv1d(aux_channels, aux_channels, kernel_size=5, padding='same'),
                nn.ReLU(),
                # nn.BatchNorm1d(aux_channels),
                nn.MaxPool1d(2)
            )
            self.conv_blocks.append(block)

        self.rnn_layer = RNNModel(
            n_input_channels=aux_channels,
            rnn_type="RNN",
            hidd_size=hidd_size,
            out_features=out_features
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.permute(0, 2, 1) 
        return self.rnn_layer(x)

class SpeechCommandTCN(nn.Module):
    def __init__(self, num_inputs, num_classes, tcn_params):
        super().__init__()
        self.tcn = TCN(num_inputs=num_inputs, **tcn_params)
        output_dim = tcn_params['num_channels'][-1]
        self.classifier = nn.Linear(output_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1) 
        out = self.tcn(x) 
        out = out.mean(dim=2) 
        return self.classifier(out)

def save_model(model, path, config=None):
    """
    saves a PyTorch model.
    Parameters:
    - model: instancia de nn.Module
    - path: destino del archivo .pt
    - config: diccionario con parámetros del modelo
    """
    os.makedirs(os.path.dirname(path), exist_ok=True)

    checkpoint = {
        "state_dict": model.state_dict(),
        "config": config if config is not None else {}
    }

    torch.save(checkpoint, path)
    print(f"Modelo guardado en {path}")

def load_trained_model(model_class, checkpoint_path, device="cuda", config=None, **override_kwargs):
    """
    loads a saved model, allowing to pass through external config
    Config priorities:
        1. Explicit config given as a parameter
        2. config guardado en el checkpoint
        2. 
        3. override_kwargs (pisan todo lo anterior)

    Parameters:
    - model_class: clase del modelo
    - checkpoint_path: ruta al archivo .pt
    - device: "cpu" o "cuda"
    - config: diccionario completo externo para construir el modelo
    - override_kwargs: parámetros puntuales que quieras reemplazar
    """

    checkpoint = torch.load(checkpoint_path, map_location=device)

    # 1) checkpoint config
    if checkpoint.get("config", {}) is not None:
        final_config = checkpoint.get("config", {}).copy()

    # 2) si el usuario entrega un config, reemplaza completamente al del checkpoint
    if config is not None:
        final_config = config.copy()

    # 3) override puntual de valores individuales
    final_config.update(override_kwargs)

    # construir modelo
    model = model_class(**final_config)

    model.load_state_dict(checkpoint["state_dict"])
    model.to(device)
    model.eval()

    print(f"Modelo cargado desde {checkpoint_path}")
    return model


## Trainers

In [ ]:
def train_step(x_batch, y_batch, model, optimizer, criterion, use_gpu):
    # Predicción
    y_predicted = model(x_batch)

    # Cálculo de loss
    loss = criterion(y_predicted, y_batch)

    # Actualización de parámetros
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return y_predicted, loss


def evaluate(val_loader, model, criterion, use_gpu):
    cumulative_loss = 0
    cumulative_predictions = 0
    data_count = 0

    for x_val, y_val in val_loader:
        if use_gpu:
            x_val = x_val.cuda()
            y_val = y_val.cuda()

        y_predicted = model(x_val)

        loss = criterion(y_predicted, y_val)

        class_prediction = torch.argmax(y_predicted, axis=1).long()

        cumulative_predictions += (y_val == class_prediction).sum().item()
        cumulative_loss += loss.item() * y_val.shape[0]
        data_count += y_val.shape[0]

    val_acc = cumulative_predictions / data_count
    val_loss = cumulative_loss / data_count

    return val_acc, val_loss


def train_model(
    model,
    train_dataset,
    val_dataset,
    epochs,
    criterion,
    batch_size,
    lr,
    n_evaluations_per_epoch=6,
    use_gpu=False,
    patience=5,                 
    model_config = None,
    model_arch = "ola"
):
    if use_gpu:
        model.cuda()

    # Dataloaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, pin_memory=use_gpu
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-9)

    curves = {"train_acc": [], "val_acc": [], "train_loss": [], "val_loss": []}

    t0 = time.perf_counter()
    iteration = 0
    n_batches = len(train_loader)

    best_val_loss = float("inf")
    epochs_without_improvement = 0

    print(n_batches)

    for epoch in range(epochs):
        print(f"\rEpoch {epoch + 1}/{epochs}")
        cumulative_train_loss = 0
        cumulative_train_corrects = 0
        examples_count = 0

        model.train()
        for i, (x_batch, y_batch) in enumerate(train_loader):
            if use_gpu:
                x_batch = x_batch.cuda()
                y_batch = y_batch.cuda()

            y_predicted, loss = train_step(x_batch, y_batch, model, optimizer, criterion, use_gpu)

            cumulative_train_loss += loss.item() * x_batch.shape[0]
            examples_count += y_batch.shape[0]

            class_prediction = torch.argmax(y_predicted, axis=1).long()
            cumulative_train_corrects += (y_batch == class_prediction).sum().item()

            if (i % (n_batches // n_evaluations_per_epoch) == 0) and (i > 0):
                train_loss = cumulative_train_loss / examples_count
                train_acc = cumulative_train_corrects / examples_count
                print(f"Iteration {iteration} - Batch {i}/{len(train_loader)} - Train loss: {train_loss}, Train acc: {train_acc}")

            iteration += 1

        with torch.no_grad():
            val_acc, val_loss = evaluate(val_loader, model, criterion, use_gpu)

        print(f"Val loss: {val_loss}, Val acc: {val_acc}")

        train_loss = cumulative_train_loss / examples_count
        train_acc = cumulative_train_corrects / examples_count

        curves["train_acc"].append(train_acc)
        curves["val_acc"].append(val_acc)
        curves["train_loss"].append(train_loss)
        curves["val_loss"].append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0

        else:
            epochs_without_improvement += 1
            print(f"Sin mejora. Paciencia: {epochs_without_improvement}/{patience}")

            if epochs_without_improvement >= patience:
                print("Early stopping activado!")
                break

    total_time = time.perf_counter() - t0
    print(f"Tiempo total de entrenamiento: {total_time:.4f} [s]")

    if model_config is not None:
        save_path = "model_weights"
        os.makedirs(save_path, exist_ok=True)
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        save_model(model, os.path.join(save_path, f"{model_arch}_{timestamp}.pt"))
    model.cpu()
    return curves, total_time



def evaluate_models_metrics(models, dataloader, criterion, use_gpu=True, num_repeats=15):
    """
    Evalúa múltiples modelos y calcula métricas promedio y desviación estándar.
    Mide el tiempo de inferencia por batch, promediando 'num_repeats' ejecuciones.
    """

    all_metrics = {
        "accuracy": [],
        "recall": [],
        "precision": [],
        "f1": [],
        "infer_time": []    # Lista de listas, donde cada lista interior son los tiempos promedio por batch
    }

    all_times = []      # ← promedio de tiempos por modelo (un valor por modelo)
    all_f1_list = []    # ← lista de f1 score por modelo (un valor por modelo)

    for model in models:
        model.eval()
        if use_gpu:
            model.cuda()

        y_true = []
        y_pred = []
        losses = []

        # Lista de tiempos de inferencia promedio (promedio de las 'num_repeats' ejecuciones)
        infer_times_per_batch = [] 

        with torch.no_grad():
            for X, y in dataloader:
                if use_gpu:
                    X, y = X.cuda(non_blocking=True), y.cuda(non_blocking=True)
                
                # Para asegurar consistencia, sincronizar GPU antes de empezar
                if use_gpu:
                    torch.cuda.synchronize()

                # ---- Medir tiempo (Inferencia Repetida) ----
                batch_run_times = []
                
                # Repetir la inferencia 'num_repeats' veces
                for r in range(num_repeats):
                    start_time = time.time()
                    outputs = model(X)
                    
                    if use_gpu:
                        torch.cuda.synchronize() # Esperar que la GPU termine
                    
                    end_time = time.time()
                    
                    # Descartar la primera ejecución (r=0) para evitar el tiempo de 'warm-up'
                    if r > 0: 
                        batch_run_times.append(end_time - start_time)

                # Calcular el tiempo de inferencia promedio para este batch
                # Si num_repeats=1, se añade 0 a batch_run_times. Hay que manejarlo.
                if len(batch_run_times) > 0:
                    avg_batch_time = np.mean(batch_run_times)
                else:
                    # Si solo se ejecutó una vez (num_repeats=1) o menos, usamos el primer tiempo.
                    # Nota: El tiempo del warm-up no es ideal, pero es el único disponible.
                    start_time = time.time()
                    outputs = model(X)
                    if use_gpu:
                        torch.cuda.synchronize()
                    avg_batch_time = time.time() - start_time
                    
                infer_times_per_batch.append(avg_batch_time)
                # --------------------------------------------
                
                loss = criterion(outputs, y)
                losses.append(loss.item())

                # Calcular predicciones y métricas
                preds = outputs.argmax(dim=1)
                y_true.extend(y.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        # --- Métricas totales por modelo ---
        acc = accuracy_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred, average='macro', zero_division='warn')
        prec = precision_score(y_true, y_pred, average='macro', zero_division='warn')
        f1 = f1_score(y_true, y_pred, average='macro', zero_division='warn')

        all_metrics["accuracy"].append(acc)
        all_metrics["recall"].append(rec)
        all_metrics["precision"].append(prec)
        all_metrics["f1"].append(f1)
        
        # Guardar los tiempos de inferencia promedio por batch para este modelo
        all_metrics["infer_time"].append(infer_times_per_batch)
        all_times.append(np.mean(infer_times_per_batch))
        all_f1_list.append(f1)

    # --- Promedios Finales ---
    # Nota: Aquí promediamos las métricas entre los modelos
    metrics_mean = {k: np.mean(v) if k != "infer_time" else None for k, v in all_metrics.items()}
    metrics_std  = {k: np.std(v)  if k != "infer_time" else None for k, v in all_metrics.items()}
    
    # El promedio y STD de "infer_time" se maneja directamente con all_times
    metrics_mean["infer_time"] = np.mean(all_times)
    metrics_std["infer_time"] = np.std(all_times)

    return metrics_mean, metrics_std, all_metrics, all_times, all_f1_list



## Visualization

In [ ]:
def plot_waveform(wf, sample_rate, label="", figname=None):
    """
    Muestra el waveform (izquierda) y los MFCCs (derecha) de una señal de audio.

    Parámetros:
        wf (Tensor): señal de audio [1, N] o [N]
        sample_rate (int): frecuencia de muestreo (Hz)
        label (str): etiqueta opcional para el título
        figname (str): ruta para guardar la figura (si es None, solo muestra)
    """
    if isinstance(wf, torch.Tensor):
        wf = wf.squeeze().cpu()

    # === Transformación MFCC ===
    mfcc_transform = torchaudio.transforms.MFCC(
        sample_rate=sample_rate,
        n_mfcc=13,
        melkwargs={"n_fft": 320, "hop_length": 160, "n_mels": 23},
        log_mels=True
    )
    mfcc = mfcc_transform(wf.unsqueeze(0)).squeeze().cpu().numpy()  # [n_mfcc, time]

    # === Crear figura con 2 subplots ===
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.set_style("whitegrid")

    # --- Waveform ---
    time = torch.arange(0, len(wf)) / sample_rate
    axes[0].plot(time, wf.numpy(), color="steelblue", linewidth=1.0)
    axes[0].set_title("Waveform", fontsize=12)
    axes[0].set_xlabel("Tiempo [s]")
    axes[0].set_ylabel("Amplitud")

    # --- MFCC ---
    sns.heatmap(mfcc, ax=axes[1], cmap="viridis", cbar=True)
    axes[1].set_title("MFCCs", fontsize=12)
    axes[1].set_xlabel("Tiempo (frames)")
    axes[1].set_ylabel("Coeficiente MFCC")

    fig.suptitle(f"Audio: {label}", fontsize=14, y=1.02)
    plt.tight_layout()

    # === Guardar o mostrar ===
    if figname:
        name = os.path.join('img', f'{figname}.pdf')
        plt.savefig(name, bbox_inches="tight")
        print(f"Figura guardada en {name}")
    else:
        plt.show()

    plt.close(fig)

 
def plot_f1_vs_time_custom_legend(df, filename='f1_vs_inference_time_shapes.pdf'):
    """
    Grafica F1 vs Tiempo.
    - Color = Arquitectura
    - Forma (Marker) = Feature
    - Leyenda personalizada dividida en dos secciones.
    """
    plt.figure(figsize=(13, 8)) # Un poco más ancho para la leyenda
    
    # 1. Definir Mapeo de Formas (Markers) para cada Feature
    # o: círculo, s: cuadrado, ^: triángulo, D: diamante, P: plus grueso
    marker_map = {
        'mfcc': 'o',              
        'mfcc_delta': 's',       
        'mfcc_delta_delta': '^', 
        'wav2vec2': 'D'          
    }
    
    # Alias para que la leyenda se vea limpia
    feat_legend_names = {
        'mfcc': 'MFCC',
        'mfcc_delta': 'MFCC Delta',
        'mfcc_delta_delta': 'MFCC Delta²',
        'wav2vec2': 'Wav2Vec2'
    }

    # 2. Definir Colores para Arquitecturas
    unique_archs = df['Architecture'].unique()

    # 3. Bucle de ploteo
    for idx, row in df.iterrows():
        arch_name = row['Architecture']
        feat_name = row['Feature']
        
        c = color_map.get(arch_name, 'gray')
        m = marker_map.get(feat_name, 'x') # 'x' por si sale una feature nueva desconocida

        plt.errorbar(
            x=row['Time (ms)'],
            y=row['F1 Mean'],
            xerr=row['Time std (ms)'],
            yerr=row['F1 Std'],
            fmt=m,               # <--- Aquí aplicamos la FORMA
            color=c,             # <--- Aquí aplicamos el COLOR
            ecolor=c,            # Color de la barra de error igual al punto
            capsize=5,
            markersize=10,       # Puntos un poco más grandes para ver la forma
            alpha=0.8,
            linestyle='None'     # Importante: que no intente unir líneas
        )

    # 4. Construcción de la Leyenda Personalizada (Manual)
    legend_elements = []
    
    # -- Sección 1: Arquitecturas (Colores) --
    legend_elements.append(Line2D([0], [0], color='w', label=r'$\bf{Arquitectura}$')) # Título falso
    for arch in unique_archs:
        legend_elements.append(
            Line2D([0], [0], marker='o', color='w', label=arch,
                   markerfacecolor=color_map[arch], markersize=10)
        )
    
    # Espacio en blanco en la leyenda
    legend_elements.append(Line2D([0], [0], color='w', label=' ')) 
    
    # -- Sección 2: Features (Formas) --
    legend_elements.append(Line2D([0], [0], color='w', label=r'$\bf{Features}$')) # Título falso
    # Filtramos solo las features que realmente existen en el dataframe
    present_features = df['Feature'].unique()
    
    for feat in present_features:
        if feat in marker_map:
            display_name = feat_legend_names.get(feat, feat)
            legend_elements.append(
                Line2D([0], [0], marker=marker_map[feat], color='w', label=display_name,
                       markerfacecolor='black', markersize=10, markeredgecolor='black')
                       # Usamos negro para indicar que la forma es lo importante, no el color
            )

    # Configuración final del gráfico
    plt.xlabel("Tiempo de inferencia (ms)", fontsize=12)
    plt.ylabel("F1 Score Promedio (%)", fontsize=12)
    plt.title("Comparativa: Arquitectura vs Feature Input", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.5)
    
    # Colocar la leyenda fuera a la derecha para no tapar datos
    plt.legend(handles=legend_elements, loc='best')
    
    plt.tight_layout()
    
    save_path = os.path.join('img', filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path)
    plt.close()
    print(f"Gráfico guardado en: {save_path}")
def show_curves(all_curves, suptitle='', filename = ''):
    min_len = {k: min(len(c[k]) for c in all_curves) for k in all_curves[0].keys()}

    trimmed = {
        k: np.array([c[k][:min_len[k]] for c in all_curves])
        for k in all_curves[0].keys()
    }

    final_curve_means = {k: trimmed[k].mean(axis=0) for k in trimmed}
    final_curve_stds  = {k: trimmed[k].std(axis=0)  for k in trimmed}

    fig, ax = plt.subplots(1, 2, figsize=(13, 5))
    fig.set_facecolor('white')

    epochs = np.arange(len(final_curve_means["val_loss"])) + 1

    ax[0].plot(epochs, final_curve_means['val_loss'], label='validation')
    ax[0].plot(epochs, final_curve_means['train_loss'], label='training')
    ax[0].fill_between(epochs,
                       y1=final_curve_means["val_loss"] - final_curve_stds["val_loss"],
                       y2=final_curve_means["val_loss"] + final_curve_stds["val_loss"], alpha=.5)
    ax[0].fill_between(epochs,
                       y1=final_curve_means["train_loss"] - final_curve_stds["train_loss"],
                       y2=final_curve_means["train_loss"] + final_curve_stds["train_loss"], alpha=.5)
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Loss evolution during training')
    ax[0].legend()

    # ==== Plot de precisión ====
    ax[1].plot(epochs, final_curve_means['val_acc'], label='validation')
    ax[1].plot(epochs, final_curve_means['train_acc'], label='training')
    ax[1].fill_between(epochs,
                       y1=final_curve_means["val_acc"] - final_curve_stds["val_acc"],
                       y2=final_curve_means["val_acc"] + final_curve_stds["val_acc"], alpha=.5)
    ax[1].fill_between(epochs,
                       y1=final_curve_means["train_acc"] - final_curve_stds["train_acc"],
                       y2=final_curve_means["train_acc"] + final_curve_stds["train_acc"], alpha=.5)
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_title('Accuracy evolution during training')
    ax[1].legend()

    fig.suptitle(suptitle, fontsize=16, weight="bold")

    filepath = os.path.join('img', f'{filename}.pdf')
    plt.savefig(filepath, bbox_inches='tight', format='pdf')
    plt.close(fig)

def get_metrics_and_confusion_matrix(models, dataset, name='', filename=''):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=min(32, len(dataset)))


    y_true = torch.cat([y for _, y in dataloader])
    total_classes = len(torch.unique(y_true))
    
    # Definir un ID seguro para 'others'
    OTHERS_ID = total_classes 

    # --- Obtener nombres de clases del dataset ---
    if hasattr(dataset, "idx_to_label"):
        # Manejo robusto si es dict o lista
        if isinstance(dataset.idx_to_label, dict):
            class_names = [dataset.idx_to_label[i] for i in range(total_classes)]
        else:
            class_names = dataset.idx_to_label
    elif hasattr(dataset, "labels"):
        class_names = dataset.labels
    elif hasattr(dataset, "classes"): # Común en ImageFolder
        class_names = dataset.classes
    else:
        class_names = [str(i) for i in range(total_classes)]

    # --- NORMALIZACIÓN DE STRINGS ---
    # Creamos un diccionario insensible a mayúsculas/espacios para buscar
    # { "stop": 3, "go": 5, ... }
    name_to_idx = {name.lower().strip(): i for i, name in enumerate(class_names)}

    target_major_names = [
        "off", "right", "stop", "go", "on", 
        "yes", "up", "down", "no", "left"
    ]
    
    # Las clases menores no necesitan listarse explícitamente para la lógica, 
    # se calculan como el complemento, pero si las usas para validar:
    # target_minor_names = [...] 

    # 2. Convertir nombres a índices (top10) con validación
    top10 = []
    found_names = []
    
    print(f"--- Validando clases para {name} ---")
    for name_query in target_major_names:
        clean_name = name_query.lower().strip()
        if clean_name in name_to_idx:
            idx = name_to_idx[clean_name]
            top10.append(idx)
            found_names.append(class_names[idx]) # Guardamos el nombre original real
        else:
            print(f"ADVERTENCIA: La clase '{name_query}' no está en el dataset.")
            print(f"   -> Clases disponibles (ejemplo): {class_names[:5]}...")
    
    top10_set = set(top10)

    # ==========================================
    # === FIN DE MODIFICACIÓN HARDCODED ===
    # ==========================================

    # --- Preparar Grupo A ---
    y_true_A = y_true.clone()
    for cls in range(total_classes):
        if cls not in top10_set:
            y_true_A[y_true_A == cls] = OTHERS_ID 

    # Usamos los nombres REALES encontrados, más 'others'
    ids_A = top10 + [OTHERS_ID] 
    label_names_A = found_names + ["others"]

    # --- Preparar Grupo B (Resto) ---
    # CORRECCIÓN: Había un error de sintaxis "not n" -> "not in"
    mask_B = torch.tensor([c.item() not in top10_set for c in y_true], dtype=torch.bool)
    
    y_true_B = y_true[mask_B]
    ids_B = sorted(torch.unique(y_true_B).tolist())
    label_names_B = [class_names[c] for c in ids_B]

    # ... (resto de tu función map_groupA, compute_group, etc.)
    def map_groupA(pred):
        predA = pred.clone()
        for cls in range(total_classes):
            if cls not in top10_set:
                predA[predA == cls] = OTHERS_ID 
        return predA

    # --- Compute Group ---
    def compute_group(models, dataloader, y_true_group, target_ids, mask=None, map_func=None, compute_f1_total=False):
        cms, f1_scores = [], []
        f1_total_scores = [] 

        for model in models:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            model.eval()

            preds_raw = [] 
            preds_group = [] 
            
            with torch.no_grad(): 
                for x, _ in dataloader:
                    x = x.to(device)
                    p = model(x).argmax(dim=1)
                    
                    preds_raw.append(p)

                    p_group = map_func(p) if map_func else p
                    preds_group.append(p_group)

            preds_raw = torch.cat(preds_raw)
            preds_group = torch.cat(preds_group)

            if mask is not None:
                preds_group = preds_group[mask]
                preds_raw = preds_raw[mask] 

            preds_cpu = preds_group.cpu()
            
            cm = confusion_matrix(
                y_true_group,
                preds_cpu,
                labels=target_ids, 
                normalize="true",
            )
            cms.append(cm)
            f1_scores.append(f1_score(y_true_group, preds_cpu, average='macro'))

            if compute_f1_total:
                f1_total = f1_score(y_true.cpu(), preds_raw.cpu(), average='macro')
                f1_total_scores.append(f1_total)

        if compute_f1_total:
            return (
                np.mean(cms, axis=0),
                np.std(cms, axis=0),
                np.mean(f1_scores) * 100,
                np.std(f1_scores) * 100,
                np.mean(f1_total_scores) * 100,
                np.std(f1_total_scores) * 100,
            )
        
        return (
            np.mean(cms, axis=0),
            np.std(cms, axis=0),
            np.mean(f1_scores) * 100,
            np.std(f1_scores) * 100,
        )

    # Llamadas a compute_group
    cmA_mean, cmA_std, accA_mean, accA_std, f1_total_mean, f1_total_std = compute_group(
        models, dataloader, y_true_A, ids_A, map_func=map_groupA, compute_f1_total=True
    )
    
    cmB_mean, cmB_std, accB_mean, accB_std = compute_group(
        models, dataloader, y_true_B, ids_B, mask=mask_B, compute_f1_total=False 
    )

    os.makedirs("img", exist_ok=True)

    def plot_both_cms(cmA_mean, cmA_std, labelsA, accA_mean, accA_std,
                            cmB_mean, cmB_std, labelsB, accB_mean, accB_std,
                            filename, f1_total_mean, f1_total_std, main_title=''): 

        vmin = min(cmA_mean.min(), cmB_mean.min())
        vmax = max(cmA_mean.max(), cmB_mean.max())
        
        if main_title:
             main_title = f"{main_title}\nOverall F1 (macro) = {f1_total_mean:.2f} ± {f1_total_std:.2f}%"
        else:
             main_title = f"Overall F1 (macro) = {f1_total_mean:.2f} ± {f1_total_std:.2f}%"

        fig, axs = plt.subplots(1, 2, figsize=(18, 8))
        fig.suptitle(main_title, fontsize=16, fontweight='bold')
        
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7]) 

        # --- Plot A ---
        ax = axs[0]
        imA = ax.imshow(cmA_mean, cmap=plt.cm.Blues, vmin=vmin, vmax=vmax)
        ax.set_title(f"Main Classes + Others\nF1 (macro): {accA_mean:.2f} ± {accA_std:.2f}%") 

        ax.set_xticks(np.arange(len(labelsA)))
        ax.set_yticks(np.arange(len(labelsA)))
        ax.set_xticklabels(labelsA, rotation=90, ha="center")
        ax.set_yticklabels(labelsA)
        
        norm = Normalize(vmin=vmin, vmax=vmax)
        for i in range(len(labelsA)):
            for j in range(len(labelsA)):
                rgba = imA.cmap(norm(cmA_mean[i, j]))
                luminance = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
                text_color = "white" if luminance < 0.5 else "black"
                ax.text(j, i, f"{cmA_mean[i,j]:.2f}\n±{cmA_std[i,j]:.2f}",
                        ha="center", va="center", fontsize=8, color=text_color)

        ax.set_xlabel("Predicted label")
        ax.set_ylabel("True label")

        # --- Plot B ---
        ax = axs[1]
        imB = ax.imshow(cmB_mean, cmap=plt.cm.Blues, vmin=vmin, vmax=vmax)
        ax.set_title(f"Minor Classes (Others breakdown)\nF1 (macro): {accB_mean:.2f} ± {accB_std:.2f}%") 

        ax.set_xticks(np.arange(len(labelsB)))
        ax.set_yticks(np.arange(len(labelsB)))
        ax.set_xticklabels(labelsB, rotation=90, ha="center")
        ax.set_yticklabels(labelsB)

        for i in range(len(labelsB)):
            for j in range(len(labelsB)):
                rgba = imB.cmap(norm(cmB_mean[i, j]))
                luminance = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
                text_color = "white" if luminance < 0.5 else "black"
                ax.text(j, i, f"{cmB_mean[i,j]:.2f}",
                        ha="center", va="center", fontsize=6, color=text_color)

        ax.set_xlabel("Predicted label")
        ax.set_ylabel("True label")

        cbar = fig.colorbar(imA, cax=cbar_ax) 
        cbar.set_label("Normalized frequency", rotation=270, labelpad=15)
        plt.subplots_adjust(right=0.9, top=0.85) 
        plt.savefig(filename, bbox_inches="tight")
        plt.close()

    plot_both_cms(
        cmA_mean, cmA_std, label_names_A, accA_mean, accA_std,
        cmB_mean, cmB_std, label_names_B, accB_mean, accB_std,
        filename=f'{filename}.pdf',
        f1_total_mean=f1_total_mean,
        f1_total_std=f1_total_std,
        main_title=f"Results for model {'s' if len(models)>1 else ''} on {name}"
    )
    
    return {
        "f1_total_mean": f1_total_mean,
        "f1_total_std": f1_total_std,
        "cmA_mean": cmA_mean,
        "cmB_mean": cmB_mean,
    }

## Feature extraction

In [ ]:
# --- Configuración base ---
ROOT_DIR = "data"
SAVE_DIR = os.path.join(ROOT_DIR, "features")
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- Parámetros comunes ---
mfcc = torchaudio.transforms.MFCC(
    sample_rate=16000,
    n_mfcc=13,
    melkwargs={"n_fft": 320, "hop_length": 160, "n_mels": 23}
)

# --- Inicializar Wav2Vec2 una sola vez ---
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)

# --- Configuración de modos y extractores ---
modes = {
    "mfcc": mfcc,
    "mfcc_delta": mfcc,
    "mfcc_delta_delta": mfcc,
    "wav2vec2": wav2vec2,
}

# --- Procesar para train y val ---
for split in ["train", "val", "test"]:
    list_path = os.path.join(ROOT_DIR, f"{split}_list.txt")

    for mode, extractor in modes.items():
        save_path = os.path.join(SAVE_DIR, f"{split}_{mode}.pt")

        if os.path.isfile(save_path):
            print(f"{save_path} ya existe, saltando...")
            continue

        print(f"\nExtrayendo {mode} para {split}...")

        dataset = CustomSpeechCommands(ROOT_DIR, list_path, mode=mode)
        if mode == "wav2vec2":
            dataset.save_features(
                feature_extractor=extractor,
                processor=processor,
                device=device,
                save_path=save_path,
            )
        else:
            dataset.save_features(
                feature_extractor=extractor,
                device=device,
                save_path=save_path,
            )

print("\nExtracción de features completada.")


In [ ]:
SAVE_DIR = os.path.join(ROOT_DIR, 'petes') # --- Procesar para train y val ---
for split in ["train", "val", "test"]:
    list_path = os.path.join(ROOT_DIR, f"{split}_list.txt")
        
    for hl in [32, 54, 320]:
        nfft = {
            32: 64,
            54: 108,
            320: 640
        }

        mode = torchaudio.transforms.MFCC(
            sample_rate=16000,
            n_mfcc=13,
            log_mels = True,
            melkwargs={"n_fft": nfft[hl], "hop_length": hl, "n_mels": 23}
        )
        save_path = os.path.join(SAVE_DIR, f"{split}_{hl}_mfcc.pt")
        if os.path.isfile(save_path):
            print(f"{save_path} ya existe, saltando...")
            continue
        print(f"\nExtrayendo {mode} para {split}...")

        dataset = CustomSpeechCommands('data', list_path, mode='mfcc')
        if mode == "wav2vec2":
            dataset.save_features(
                feature_extractor=extractor,
                processor=processor,
                device=device,
                save_path=save_path,
            )
        else:
            dataset.save_features(
                feature_extractor=mode,
                device=device,
                save_path=save_path,
            )

print("\nExtracción de features completada.")

## Training

In [ ]:
ROOT_DIR = os.path.join("data", "petes")
SAVE_DIR = ROOT_DIR
MODEL_WEIGHT_PATH = 'seq_length_model_weights'
device = "cuda" if torch.cuda.is_available() else "cpu"

# Configuración TCN
tcn_config = {
    'num_channels': [64, 64, 128, 128],
    'kernel_size': 3,
    'dilations': [1, 2, 4, 8],
    'dropout': 0,
    'causal': False,
    'input_shape': 'NCL',
    'use_norm': 'batch_norm',
    'use_skip_connections': True
}

lr = 5e-4
batch_size = 32
criterion = nn.CrossEntropyLoss()
n_trains = 5
epochs = 30
use_gpu = True
architectures = ["GRU", "LSTM", "RNN", "CNN", "TCNN", "Transformer"]

# Inicializar diccionarios
f1_scores_seq_len = {arch: [] for arch in architectures}
f1_stds_seq_len = {arch: [] for arch in architectures}

hop_lengths = [1600, 800, 320, 160, 54, 32]
seq_len_seen = []

# --- Inicio del Loop Principal ---
for hop_length in hop_lengths:
    # 1. Calcular largo de secuencia
    seq_len_input = 1 + 16000 // hop_length
    seq_len_seen.append(seq_len_input)
    print(f"\n>>> PROCESANDO HOP: {hop_length} | SEQ LEN: {seq_len_input}")

    # 2. Cargar Datasets
    # Nota: Asegúrate de que existan. Si fallan, revisa tus rutas.
    train_path = os.path.join(SAVE_DIR, f"train_{hop_length}_mfcc.pt")
    val_path   = os.path.join(SAVE_DIR, f"val_{hop_length}_mfcc.pt")
    test_path  = os.path.join(SAVE_DIR, f"test_{hop_length}_mfcc.pt")

    train_dataset = FeaturesDataset(train_path)
    val_dataset   = FeaturesDataset(val_path)
    test_dataset  = FeaturesDataset(test_path)
    
    # Creamos el loader de test una sola vez por hop_length
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    for arch in architectures:
        print(f"   --- Arquitectura: {arch} ---")

        models = []
        curves = [] # Para guardar curvas de los que se entrenen en esta corrida

        for k in range(n_trains):
            # A. Instanciar Modelo
            if arch in ["GRU", "LSTM", "RNN"]:
                config = {'rnn_type': arch, 'n_input_channels': 13, 'hidd_size': 256, 'out_features': 35, 'num_layers': 1}
                model = RNNModel(**config)
            elif arch == "CNN":
                config = {'n_input_channels': 13, 'hidd_size': 64, 'out_features': 35}
                model = CNNModel(**config)
            elif arch == "TCNN":
                config = {'num_inputs': 13, 'num_classes': 35, 'tcn_params': tcn_config}
                model = SpeechCommandTCN(**config)
            elif arch == 'Transformer':
                config = {'n_input_features': 13, 'n_output_classes': 35, 'd_model': 128, 'nhead': 8, 'd_hid': 512, 'n_layers': 4, 'dropout': 0.0}
                model = TransformerModel(**config)
            
            # B. Definir Ruta
            filename = f'{arch}_{k+1}_train_seq_len_[{seq_len_input}].pt'
            filepath = os.path.join(MODEL_WEIGHT_PATH, arch, filename)

            # C. Cargar o Entrenar (Lógica Corregida)
            if os.path.exists(filepath):
                print(f"      [SKIP] Cargando {filename}...")
                loaded_data = torch.load(filepath)
                # Manejo de versiones de guardado (por si guardaste todo el dict o solo state_dict)
                if 'state_dict' in loaded_data:
                    model.load_state_dict(loaded_data['state_dict'])
                else:
                    model.load_state_dict(loaded_data)
            else:
                print(f"      [TRAIN] Entrenando {filename}...")
                
                # 1. Entrenar UNA VEZ
                curve, _ = train_model(
                    model, train_dataset, val_dataset, epochs, criterion, batch_size, lr,
                    n_evaluations_per_epoch=3, use_gpu=use_gpu, patience=10,
                    model_config=config, model_arch=arch
                )
                curves.append(curve)
                
                # 2. Guardar UNA VEZ
                os.makedirs(os.path.dirname(filepath), exist_ok=True)
                save_model(model, filepath, config)
        
            # D. Agregar a la lista de modelos para evaluar (importante: fuera del if/else)
            models.append(model)

        # Fin del loop k (5 modelos cargados/entrenados)

        # Opcional: Mostrar curvas solo si hubo entrenamiento nuevo
        if len(curves) > 0:
            show_curves(
                curves,
                suptitle=f"{arch} - Seq Len {seq_len_input}",
                filename=os.path.join(arch, f"{arch}_seq_len{seq_len_input}")
            )

        # 3. Evaluar el conjunto (Ensemble o promedio de métricas)
        # Usamos la función corregida con ddof=1 (si la actualizaste)
        metrics_mean, metrics_std, *_ = evaluate_models_metrics(models, test_loader, criterion, use_gpu=use_gpu)

        f1_scores_seq_len[arch].append(metrics_mean["f1"])
        f1_stds_seq_len[arch].append(metrics_std["f1"])

        # Matriz de Confusión
        img_conf_dir = os.path.join('img', arch)
        os.makedirs(img_conf_dir, exist_ok=True)
        
        get_metrics_and_confusion_matrix(
            models,
            test_dataset,
            name=f"{arch} (Seq Len: {seq_len_input})", 
            filename=os.path.join(img_conf_dir, f"conf_mat_{arch}_len{seq_len_input}.pdf")
        )

# --- Gráfico Final ---
print("\nGenerando gráfico comparativo final...")

# Definimos mapa de colores si no existe

plt.figure(figsize=(10, 6))

for arch in architectures:
    # Aseguramos que x e y tengan el mismo largo antes de plotear
    if len(f1_scores_seq_len[arch]) == len(seq_len_seen):
        plt.errorbar(
            seq_len_seen, 
            f1_scores_seq_len[arch], 
            yerr=f1_stds_seq_len[arch], 
            marker="o", 
            capsize=4, 
            label=arch,
            alpha=0.8,
            color=color_map.get(arch, 'black')
        )

plt.xlabel("Largo de secuencia de entrada (frames)")
plt.ylabel("F1-score promedio (Macro)")
plt.title("Performance vs Largo de Secuencia")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()

out_plot_path = os.path.join("img", "f1_vs_seq_len_comparison.pdf")
os.makedirs(os.path.dirname(out_plot_path), exist_ok=True)
plt.savefig(out_plot_path, bbox_inches="tight")
plt.close()

print(f"Proceso completado. Gráfico guardado en {out_plot_path}")

In [ ]:
n_trains = 5
epochs = 30
batch_size = 32
lr = 5e-4
criterion = nn.CrossEntropyLoss()
use_gpu = True
MODEL_WEIGHT_PATH = 'cnn_rnn_weights'
models = []
curves = []


for k in range(n_trains):
    arch = 'CNNRNN'
    filename = f'{arch}_N_{k}.pt'
    print(f"\n===== Entrenamiento {k+1}/{n_trains} =====")

    config = {
        'hidd_size': 256,
        'in_channels': 1,
        'conv_out_channels': 128,
        'out_features': 35,
        'N_conv_blocks': 4
    }
    model = CNN1DModel(**config)

    filepath = os.path.join(MODEL_WEIGHT_PATH, arch, filename)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    # -------------------------------------
    # Cargar pesos si ya existe el archivo
    # -------------------------------------
    if os.path.exists(filepath):
        print(f"[SKIP] Cargando pesos existentes: {filename}")
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint["state_dict"])
        curve = checkpoint.get("curve", None)
    else:

        curve, _ = train_model(
            model,
            train_dataset,
            val_dataset,
            epochs,
            criterion,
            batch_size,
            lr,
            n_evaluations_per_epoch=3,
            use_gpu=use_gpu,
            patience=10,
            model_config=config,
            model_arch=arch,
        )

        save_model(model, filepath, config)

    curves.append(curve)
    models.append(model)

# --------------------------------------------
# EVALUAR ESTA ARQUITECTURA COMPLETA (5 modelos)
# --------------------------------------------
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)
metrics_mean, metrics_std, *_ = evaluate_models_metrics(
    models, test_loader, criterion, use_gpu=use_gpu
)

# MATRIZ DE CONFUSIÓN
get_metrics_and_confusion_matrix(
    models,
    test_dataset,
    name=f"{arch}",
    filename=os.path.join("img", arch, f"conf_mat_{arch}")
)


In [ ]:
ROOT_DIR = os.path.join("data","features")
SAVE_DIR = ROOT_DIR
device = "cuda"
MODEL_WEIGHT_PATH = 'diff_features'

device = "cuda"

tcn_config = {
    'num_channels': [64, 64, 128, 128],
    'kernel_size': 3,
    'dilations': [1, 2, 4, 8],
    'dropout': 0,
    'causal': False,             # False es mejor para clasificación offline
    'input_shape': 'NCL',        # Changed to NCL as input is permuted to (Batch, Features, Time)
    'use_norm': 'batch_norm',            # Set to None as per instruction
    'use_skip_connections': True # Keep skip connections enabled
}
# === FEATURES A USAR ===
feature_sets = [
    "mfcc",
    "mfcc_delta",
    "mfcc_delta_delta",
    "wav2vec2"
]

architectures = ["GRU", "LSTM", "RNN", "CNN", "TCNN", "Transformer"]
color_map = {
    arch: default_colors[i % len(default_colors)]
    for i, arch in enumerate(architectures)
}

n_trains = 5
epochs = 30
batch_size = 32
lr = 5e-4
criterion = nn.CrossEntropyLoss()
use_gpu = True

f1_scores = {arch: {} for arch in architectures}
f1_stds   = {arch: {} for arch in architectures}

for feat in feature_sets:
    print(f"\n\n=======================")
    print(f"=== PROBANDO FEATURE: {feat} ===")
    print(f"=======================\n")

    # Rutas según feature
    train_path = os.path.join(SAVE_DIR, f"train_{feat}.pt")
    val_path   = os.path.join(SAVE_DIR, f"val_{feat}.pt")
    test_path  = os.path.join(SAVE_DIR, f"test_{feat}.pt")

    # Cargar datasets
    train_dataset = FeaturesDataset(train_path)
    val_dataset   = FeaturesDataset(val_path)
    test_dataset  = FeaturesDataset(test_path)

    # Determinar dims usando un batch
    sample = train_dataset[0][0]
    input_features = sample.shape[-1]  # canales reales del feature
    seq_len_input = sample.shape[0]

    print(f"Dimensiones detectadas para {feat}: seq_len={seq_len_input}, features={input_features}")

    # Crear dataloader test
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    for arch in architectures:
        print(f"\n======= Entrenando arquitectura {arch} con {feat} =======\n")

        models = []
        curves = []

        for k in range(n_trains):

            # -------------------------
            # CREAR MODELO SEGÚN TIPO
            # -------------------------
            if arch in ["GRU", "LSTM", "RNN"]:
                config = {
                    'rnn_type': arch,
                    'n_input_channels': input_features,
                    'hidd_size': 256,
                    'out_features': 35,
                    'num_layers': 1
                }
                model = RNNModel(**config)

            elif arch == "CNN":
                config = {
                    'n_input_channels': input_features,
                    'hidd_size': 64,
                    'out_features': 35
                }
                model = CNNModel(**config)

            elif arch == "TCNN":
                tcn_config['num_channels'][0] = 64  # opcional
                config = {
                    'num_inputs': input_features,
                    'num_classes': 35,
                    'tcn_params': tcn_config
                }
                model = SpeechCommandTCN(**config)

            elif arch == "Transformer":
                config = {
                    'n_input_features': input_features,
                    'n_output_classes': 35,
                    'd_model': 128,
                    'nhead': 8,
                    'd_hid': 512,
                    'n_layers': 4,
                    'dropout': 0.0
                }
                model = TransformerModel(**config)

            # -------------------------
            # CARGA O ENTRENAMIENTO
            # -------------------------
            filename = f"{arch}_{feat}_{k+1}.pt"
            filepath = os.path.join(MODEL_WEIGHT_PATH, arch, feat, filename)

            if os.path.exists(filepath):
                print(f"[SKIP] Cargando pesos existentes: {filename}")
                model.load_state_dict(torch.load(filepath)["state_dict"])

            else:
                print(f"[TRAIN] Entrenando modelo {arch} ({k+1}/5) con {feat}")
                curve, _ = train_model(
                    model,
                    train_dataset,
                    val_dataset,
                    epochs,
                    criterion,
                    batch_size,
                    lr,
                    n_evaluations_per_epoch=3,
                    use_gpu=use_gpu,
                    patience=10,
                    model_config=config,
                    model_arch=arch
                )
                curves.append(curve)

                os.makedirs(os.path.dirname(filepath), exist_ok=True)
                save_model(model, filepath, config)

            models.append(model)

        # -------------------------
        # EVALUAR ESTA ARQUITECTURA
        # -------------------------
        metrics_mean, metrics_std, *_ = evaluate_models_metrics(
            models, test_loader, criterion, use_gpu=use_gpu
        )

        f1_scores[arch][feat] = metrics_mean["f1"]
        f1_stds[arch][feat]   = metrics_std["f1"]

        # MATRIZ DE CONFUSIÓN
        get_metrics_and_confusion_matrix(
            models,
            test_dataset,
            name=f"{arch} usando {feat}",
            filename=os.path.join("img", arch, f"conf_mat_{arch}_{feat}")
        )


In [ ]:
show_curves(curves, 'CNN+RNN', 'cnn_RNN.pdf')

In [ ]:

for feat in feature_sets:
    max_feat_perf = 0.0
    best_arch = ''
    for arch in architectures:
        f1 = f1_scores[arch][feat]
        if max_feat_perf < f1:
            max_feat_perf = f1
            best_arch = arch
    print(f'Best model for {feat}: {best_arch} -> f1 = {max_feat_perf*100:.2f}% ± {f1_stds[best_arch][feat]*100:.2f}%')

In [ ]:
x = np.arange(len(feature_sets))

plt.figure(figsize=(12, 7))

for arch in architectures:
    # Obtener arrays de mean y std en el mismo orden de features
    means = np.array([f1_scores[arch][feat] for feat in feature_sets])
    stds  = np.array([f1_stds[arch][feat]   for feat in feature_sets])

    plt.errorbar(x, means, yerr=stds,label=arch, marker='o', capsize=4, alpha=0.8, color = color_map[arch])

FIG_DIR = 'img'
plt.xticks(x, feature_sets, rotation=20)
plt.xlabel("Feature usado")
plt.ylabel("F1-score")
plt.title("F1 vs Feature para cada arquitectura (con desviación estándar)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'f1_vs_feats.pdf'))
plt.close()



In [ ]:
x = np.arange(len(feature_sets))

plt.figure(figsize=(12, 7))

for arch in architectures:
    if arch == 'RNN':
        continue
    # Obtener arrays de mean y std en el mismo orden de features
    means = np.array([f1_scores[arch][feat] for feat in feature_sets])
    stds  = np.array([f1_stds[arch][feat]   for feat in feature_sets])

    plt.errorbar(x, means, yerr=stds,label=arch, marker='o', capsize=4, alpha=0.8, color = color_map[arch])

FIG_DIR = 'img'
plt.xticks(x, feature_sets, rotation=20)
plt.xlabel("Feature usado")
plt.ylabel("F1-score")
plt.title("F1 vs Feature para cada arquitectura (con desviación estándar)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'f1_vs_feats_skip_rnn.pdf'))
plt.close()

In [ ]:
# --- Gráfico 1: Todas las arquitecturas ---
plt.figure(figsize=(10, 6))

for arch in architectures:
    plt.errorbar(
        seq_len_seen, 
        f1_scores_seq_len[arch], 
        yerr=f1_stds_seq_len[arch], 
        marker="o", 
        capsize=4, 
        label=arch,
        alpha=0.8,
        color = color_map[arch]
    )

# --- NUEVO: Etiquetas de valor X encima de los puntos ---
for i, x_val in enumerate(seq_len_seen):
    # Buscamos el Y máximo (score + std) en este punto X entre TODAS las arquitecturas
    # para que el texto quede arriba de la línea más alta y no se solape.
    max_y_at_x = max([f1_scores_seq_len[arch][i] + f1_stds_seq_len[arch][i] for arch in architectures])
    
    plt.text(
        x_val, 
        max_y_at_x + 0.01,  # Offset vertical para que no toque el punto
        str(x_val), 
        ha='center', 
        va='bottom', 
        fontsize=9, 
        fontweight='bold'
    )
# --------------------------------------------------------

plt.xscale('log')
plt.xlabel("Largo de secuencia de entrada (frames)")
plt.ylabel("F1-score promedio (± std)")
plt.title("Performance vs Largo de Secuencia por Arquitectura")
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("img/f1_vs_seq_len_comparison.pdf", bbox_inches="tight")
plt.close()


# --- Gráfico 2: Sin RNN ---
plt.figure(figsize=(10, 6))

active_archs = [a for a in architectures if a != 'RNN'] # Filtramos la lista antes

for arch in active_archs:
    plt.errorbar(
        seq_len_seen, 
        f1_scores_seq_len[arch], 
        yerr=f1_stds_seq_len[arch], 
        marker="o", 
        capsize=4, 
        label=arch,
        alpha=0.8,
        color = color_map[arch]
    )

# --- NUEVO: Etiquetas ---
for i, x_val in enumerate(seq_len_seen):
    # Calculamos el máximo solo sobre las arquitecturas activas (Sin RNN)
    max_y_at_x = max([f1_scores_seq_len[arch][i] + f1_stds_seq_len[arch][i] for arch in active_archs])
    
    plt.text(
        x_val, 
        max_y_at_x + 0.01, 
        str(x_val), 
        ha='center', 
        va='bottom', 
        fontsize=9,
        fontweight='bold'
    )
# ----------------------
plt.xscale('log')
plt.xlabel("Largo de secuencia de entrada (frames)")
plt.ylabel("F1-score promedio (± std)")
plt.title("Performance vs Largo de Secuencia por Arquitectura")
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("img/f1_vs_seq_len_comparison_rnn_skip.pdf", bbox_inches="tight")
plt.close()


# --- Gráfico 3: Solo RNN ---
plt.figure(figsize=(10, 6))

# Solo hay una arquitectura aquí, pero mantenemos la lógica por consistencia
target_arch = 'RNN' 

if target_arch in architectures: # Chequeo de seguridad
    plt.errorbar(
        seq_len_seen, 
        f1_scores_seq_len[target_arch], 
        yerr=f1_stds_seq_len[target_arch], 
        marker="o", 
        capsize=4, 
        label=target_arch,
        alpha=0.8,
        color = color_map[target_arch]
    )

    # --- NUEVO: Etiquetas ---
    for i, x_val in enumerate(seq_len_seen):
        # Máximo solo de RNN
        val_y = f1_scores_seq_len[target_arch][i] + f1_stds_seq_len[target_arch][i]
        
        plt.text(
            x_val, 
            val_y + 0.01, 
            str(x_val), 
            ha='center', 
            va='bottom', 
            fontsize=9,
            fontweight='bold'
        )
    # ----------------------

plt.xlabel("Largo de secuencia de entrada (frames)")
plt.ylabel("F1-score promedio (± std)")
plt.title("Performance vs Largo de Secuencia por Arquitectura")
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("img/f1_vs_seq_len_comparison_rnn_only.pdf", bbox_inches="tight")
plt.close()

print(f1_stds_seq_len)

In [ ]:
num_seq_lens = len(next(iter(f1_scores_seq_len.values())))

for i in range(num_seq_lens):
    max_f1 = -1
    best_arch = None

    for arch in architectures:
        f1_val = f1_scores_seq_len[arch][i]       # f1 de esta arch para este seq_len
        if f1_val > max_f1:
            max_f1 = f1_val
            best_arch = arch

    std_val = f1_stds_seq_len[best_arch][i]

    print(f'SeqLen {i}: {best_arch} -> f1 = {max_f1*100:.2f}% ± {std_val*100:.2f}%')


In [ ]:
# Encabezado
header = f"{'Seq Len':<10} | " + " | ".join([f"{arch:^16}" for arch in architectures])
print("-" * len(header))
print(header)
print("-" * len(header))

for i in range(num_seq_lens):
    current_seq_len = seq_len_seen[i] if 'seq_len_seen' in locals() else i
    
    # Calcular mejor score para marcarlo
    scores_at_i = [f1_scores_seq_len[arch][i] for arch in architectures]
    max_score = max(scores_at_i)
    
    row_str = f"{str(current_seq_len):<10} | "
    
    for arch in architectures:
        mean = f1_scores_seq_len[arch][i] * 100
        std = f1_stds_seq_len[arch][i] * 100
        
        # Si es el mejor, lo ponemos en negrita (código ANSI) o con un *
        is_best = (f1_scores_seq_len[arch][i] == max_score)
        marker = "*" if is_best else " "
        
        val_str = f"{mean:.2f}±{std:.2f}{marker}"
        row_str += f"{val_str:^16} | "
        
    print(row_str)

In [ ]:
device = "cuda"
def infer_model_type(path):
    name = path.lower()
    if "gru" in name:
        return "GRU"
    if "lstm" in name:
        return "LSTM"
    if "rnn" in name and "cnn" not in name:
        return "RNN"
    if "tcnn" in name:
        return "TCNN"
    if "transformer" in name:
        return "TRANSFORMER"
    raise ValueError(f"No pude inferir tipo de modelo desde: {path}")

MODEL_CLASS_BY_TYPE = {
    "GRU": RNNModel,
    "LSTM": RNNModel,
    "RNN": RNNModel,
    "TCNN": CNNModel,
    "TRANSFORMER": TransformerModel,
}

def load_model_by_type(model_path, device="cuda", config_override=None):
    """
    Carga un modelo según su tipo (GRU, LSTM, RNN, TCNN, TRANSFORMER)
    y permite pasar un config explícito.
    """
    model_type = infer_model_type(model_path)

    # si se entrega config explícito → úsalo
    if config_override is not None:
        return load_trained_model(
            MODEL_CLASS_BY_TYPE[model_type],
            model_path,
            device=device,
            config=config_override
        )

    # configs por defecto según tipo
    if model_type in ["GRU", "LSTM", "RNN"]:
        cfg = {
            'rnn_type': model_type,
            'n_input_channels': 768,
            'hidd_size': 256,
            'out_features': 35,
            'num_layers': 1
        }
        return load_trained_model(RNNModel, model_path, device=device, config=cfg)

    elif model_type == "TCNN":
        cfg = {
            'num_inputs': 768,
            'out_features': 35
        }
        return load_trained_model(SpeechCommandTCN, model_path, device=device, config=cfg)

    elif model_type == "TRANSFORMER":
        return load_trained_model(TransformerModel, model_path, device=device)

    else:
        raise ValueError(model_type)

model_paths = sorted(glob.glob("old_model_weights/*.pt"))

# Skippear transformers antes de procesar
model_paths = [p for p in model_paths if infer_model_type(p) != "TRANSFORMER"]

# Diccionario para guardar modelos por arquitectura
models_by_arch = {}
paths_by_arch = {}

for path in model_paths:
    arch = infer_model_type(path)

    model = load_model_by_type(path, device="cuda")
    models_by_arch.setdefault(arch, []).append(model)
    paths_by_arch.setdefault(arch, []).append(path)

print("Modelos cargados por arquitectura:")
for arch, lst in models_by_arch.items():
    print(f"{arch}: {len(lst)} modelos")

results_by_arch = {}
times_by_arch = {}
f1_dists_by_arch = {}


# batch_size = 32
# criterion = nn.CrossEntropyLoss()
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# for arch, model_list in models_by_arch.items():
#     print(f"\nEvaluando arquitectura: {arch}")

#     metrics_mean, metrics_std, all_metrics, mean_times, f1_dists = \
#         evaluate_models_metrics(model_list, test_loader, criterion)

    

#     results_by_arch[arch] = {
#         "mean": metrics_mean,
#         "std": metrics_std,
#         "all": all_metrics,
#     }
#     print(results_by_arch[arch])

#     # Guardamos tiempos y distribuciones f1
#     times_by_arch[arch] = mean_times
#     f1_dists_by_arch[arch] = f1_dists

# # ============================================================
# # 1) Calcular métricas agregadas por arquitectura
# # ============================================================

os.makedirs("img", exist_ok=True)

for arch, model_list in models_by_arch.items():
    print(f"\n=== Arquitectura: {arch} ===")

    # Path base para guardar el PDF
    outfile = os.path.join("img", f"conf_mat_{arch}.pdf")

    # Llamar directamente a tu función tal como existe
    metrics = get_metrics_and_confusion_matrix(
        model_list,
        test_dataset,
        name=outfile
    )



# arch_names = []
# arch_f1_values = []
# arch_f1_dists = []
# arch_time_mean = []
# arch_time_std = []

# for arch in models_by_arch.keys():
#     arch_names.append(arch)

#     # Distribuciones F1 (1 valor por modelo)
#     f1_vals = results_by_arch[arch]["all"]["f1"]
#     arch_f1_values.append(np.mean(f1_vals))
#     arch_f1_dists.append(f1_vals)

#     tvals = times_by_arch[arch]     
#     arch_time_mean.append(np.mean(tvals))
#     arch_time_std.append(np.std(tvals))


# arch_time_mean_ms = [t * 100000 for t in arch_time_mean]
# arch_time_std_ms  = [s * 100000 for s in arch_time_std]

# plot_f1_vs_inference_time_with_error_bars(
#     arch_names,
#     arch_time_mean_ms,
#     arch_f1_dists,
#     time_stds=arch_time_std_ms
# )


In [ ]:

# ==========================================
# 0. CONFIGURACIÓN TCNN (Necesaria para recrear la arquitectura)
# ==========================================
tcn_config_params = {
    'num_channels': [64, 64, 128, 128],
    'kernel_size': 3,
    'dilations': [1, 2, 4, 8],
    'dropout': 0,
    'causal': False,
    'input_shape': 'NCL',
    'use_norm': 'batch_norm',
    'use_skip_connections': True
}

# ==========================================
# 1. CONFIGURACIÓN Y MAPEOS
# ==========================================
ROOT_DIR = "diff_features"
device = "cuda" if torch.cuda.is_available() else "cpu"

FEATURE_DIMS = {
    "mfcc": 13,
    "mfcc_delta": 26,       
    "mfcc_delta_delta": 39, 
    "wav2vec2": 768
}

MODEL_CLASS_BY_TYPE = {
    "GRU": RNNModel,
    "LSTM": RNNModel,
    "RNN": RNNModel,
    "TCNN": SpeechCommandTCN, 
    "CNN": CNNModel,
    "TRANSFORMER": TransformerModel,
}

def infer_model_type(path):
    name = path.lower()
    if "gru" in name: return "GRU"
    if "lstm" in name: return "LSTM"
    if "rnn" in name: return "RNN"
    if "tcnn" in name: return "TCNN"
    if "cnn" in name: return "CNN"
    if "transformer" in name: return "TRANSFORMER"
    return "UNKNOWN"

def load_model_dynamic(model_path, input_dim, device="cuda"):
    """
    Carga el modelo inyectando la dimensión de entrada correcta.
    """
    model_type = infer_model_type(model_path)
    model_class = MODEL_CLASS_BY_TYPE[model_type]
    
    config = {}
    
    # --- CONFIGURACIÓN ESPECÍFICA POR ARQUITECTURA ---
    if model_type in ["GRU", "LSTM", "RNN"]:
        config = {
            'rnn_type': model_type,
            'n_input_channels': input_dim, 
            'hidd_size': 256,
            'out_features': 35,
            'num_layers': 1
        }
    elif model_type == "CNN":
        config = {
            'n_input_channels': input_dim, 
            'hidd_size': 64,
            'out_features': 35
        }
    
    # CORRECCIÓN TCNN: Usar las llaves correctas para SpeechCommandTCN
    elif model_type == "TCNN":
        config = {
            'num_inputs': input_dim,      # TCNN usa 'num_inputs', no 'n_input_channels'
            'num_classes': 35,
            'tcn_params': tcn_config_params
        }
    
    elif model_type == "TRANSFORMER":
        config = {
            'n_input_features': input_dim,
            'n_output_classes': 35, 
            'd_model': 128, 
            'nhead': 8, 
            'd_hid': 512, 
            'n_layers': 4
        }

    return load_trained_model(model_class, model_path, device=device, config=config)

# ==========================================
# 2. BUCLE PRINCIPAL
# ==========================================

architectures_inf = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]
metric_results_inf = []

print(f"Arquitecturas encontradas: {architectures_inf}")

for arch in architectures_inf:
    arch_path = os.path.join(ROOT_DIR, arch)
    
    features_inf = [d for d in os.listdir(arch_path) if os.path.isdir(os.path.join(arch_path, d))]
    
    for feat_inf in features_inf: # <--- Usamos feat_inf
        
        # CORRECCIÓN: Usar feat_inf para construir el path, NO 'feat'
        feat_path = os.path.join(arch_path, feat_inf)
        
        # Obtener dimensión esperada
        input_dim = FEATURE_DIMS.get(feat_inf)
        
        if input_dim is None:
            print(f"[SKIP] Dimensión desconocida para feature: {feat_inf}. Saltando...")
            continue

        # 1. Buscar los 5 modelos .pt
        model_files = glob.glob(os.path.join(feat_path, "*.pt"))
        model_files.sort()
        
        if not model_files:
            continue

        print(f"\n>>> Evaluando: {arch} | Feature: {feat_inf} | Input Dim: {input_dim}")
        print(f"    Modelos encontrados: {len(model_files)}")

        # 2. Cargar Dataset de Test CORRESPONDIENTE
        # CORRECCIÓN: Usar feat_inf aquí también
        test_dataset_path = os.path.join("data", "features", f"test_{feat_inf}.pt") 
        
        if not os.path.exists(test_dataset_path):
            # Intento de fallback por si tus archivos se llaman distinto (ej: mfcc_delta -> mfcc)
            print(f"    [ALERTA] No existe {test_dataset_path}.")
            continue
            
        test_dataset_inf = FeaturesDataset(test_dataset_path)
        test_loader_inf = torch.utils.data.DataLoader(test_dataset_inf, batch_size=32, shuffle=False)

        # 3. Cargar Modelos
        models_list = []
        for m_path in model_files:
            try:
                m = load_model_dynamic(m_path, input_dim, device=device)
                models_list.append(m)
            except Exception as e:
                print(f"    [ERROR CRÍTICO] Falló carga de {os.path.basename(m_path)}: {e}")

        if not models_list:
            print("    No se cargaron modelos válidos. Saltando evaluación.")
            continue

        # 4. Evaluar
        metrics_mean_inf, metrics_std_inf, _, _, _ = evaluate_models_metrics(
            models_list, test_loader_inf, nn.CrossEntropyLoss(), use_gpu=(device=="cuda")
        )
        
        # 5. Guardar Resultados
        row = {
            "Architecture": arch,
            "Feature": feat_inf,
            "F1 Mean": metrics_mean_inf["f1"] * 100,
            "F1 Std": metrics_std_inf["f1"] * 100,
            "Acc Mean": metrics_mean_inf["accuracy"] * 100,
            "Time (ms)": metrics_mean_inf["infer_time"]*1000,
            "Time std (ms)": metrics_std_inf['infer_time']*1000
        }
        metric_results_inf.append(row)
        
        # Matriz de confusión (Opcional)
        img_name = f"conf_mat_{arch}_{feat_inf}.pdf"
        save_path = os.path.join("img", "confusion_matrices", img_name)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Descomenta si quieres generar las imágenes
        # get_metrics_and_confusion_matrix(models_list, test_dataset_inf, name=f"{arch} - {feat_inf}", filename=save_path)

# ==========================================
# 3. VISUALIZAR TABLA FINAL
# ==========================================
if metric_results_inf:
    df_final = pd.DataFrame(metric_results_inf)
    plot_f1_vs_time_from_df(df_final, filename='comparativa_modelos_f1_tiempo.pdf')
    df_final["F1 Score"] = df_final.apply(lambda x: f"{x['F1 Mean']:.2f} ± {x['F1 Std']:.2f}", axis=1)
    df_final = df_final.sort_values(["Architecture", "F1 Mean"], ascending=[True, False])

    print("\n=== TABLA RESUMEN DE RESULTADOS ===")
    print(df_final[["Architecture", "Feature", "F1 Score", "Time (ms)", "Time std (ms)"]])
else:
    print("No se generaron resultados.")


In [ ]:


plot_f1_vs_time_custom_legend(df_final, filename='comparativa_modelos_f1_tiempo.pdf')